# 챗봇

In [2]:
# 공통코드
import numpy as np
import pandas as pd


## 데이터 가져오기

In [4]:
#corpus = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')
corpus = pd.read_csv('./ChatbotData.csv')

# 2,000개 데이터 셋만 활용 (Google Colab 일 경우 3,000개에서는 메모리 오버되는 현상 발생)
texts = []
pairs = []
for i, (text, pair) in enumerate(zip(corpus['Q'], corpus['A'])):
    texts.append(text)
    pairs.append(pair)
    
    #메모리가 부족하면 데이터 개수 조절
    if i >= 3000: 
        break 

In [5]:
# question와 answer 데이터 확인
list(zip(texts, pairs))[0:10]

[('12시 땡!', '하루가 또 가네요.'),
 ('1지망 학교 떨어졌어', '위로해 드립니다.'),
 ('3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('PPL 심하네', '눈살이 찌푸려지죠.'),
 ('SD카드 망가졌어', '다시 새로 사는 게 마음 편해요.'),
 ('SD카드 안돼', '다시 새로 사는 게 마음 편해요.'),
 ('SNS 맞팔 왜 안하지ㅠㅠ', '잘 모르고 있을 수도 있어요.'),
 ('SNS 시간낭비인 거 아는데 매일 하는 중', '시간을 정하고 해보세요.'),
 ('SNS 시간낭비인데 자꾸 보게됨', '시간을 정하고 해보세요.')]

## 데이터 전처리

In [9]:
import re
def clean_sentence(sentence):
#     # 한글, 숫자, 영문 대/소문자를 제외한 모든 문자는 제거합니다.
#     sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣ㅣa-zA-Z ]',r'', sentence)
    # 한글, 숫자를 제외한 모든 문자는 제거합니다.
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣]',r'', sentence)
    return sentence

# 전처리 함수 테스트
print(clean_sentence('안녕하세요~:)'))
print(clean_sentence('TensorFlow^@^%#@!'))

안녕하세요



In [10]:
#한글 형태소 분석
from konlpy.tag import Okt
okt = Okt()
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))

In [11]:
#챗봇을 위해 필요한 데이터 생성 함수
def clean_and_morph(sentence, is_question=True):
    # 한글 문장 전처리
    sentence = clean_sentence(sentence)
    # 형태소 변환
    sentence = process_morph(sentence)
    # Question 인 경우, Answer인 경우를 분기하여 처리
    # Answer에는 시작 과 종료 기호 추가
    if is_question:
        return sentence
    else:
        # START 토큰은 decoder input에 END 토큰은 decoder output에 추가합니다.
        return ('<START> ' + sentence, sentence + ' <END>')

In [12]:
#챗봇을 위한 데이터 생성
def preprocess(texts, pairs):
    questions = []
    answer_in = []
    answer_out = []

    # 질의에 대한 전처리
    for text in texts:
        # 전처리와 morph 수행
        question = clean_and_morph(text, is_question=True)
        questions.append(question)

    # 답변에 대한 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)
    
    return questions, answer_in, answer_out

In [21]:
#챗봇 훈련에 필요한 데이터 생성 및 확인
questions, answer_in, answer_out = preprocess(texts, pairs)

print(questions[:2])
print(answer_in[:2])
print(answer_out[:2])

['12시 땡', '1 지망 학교 떨어졌어']
['<START> 하루 가 또 가네요', '<START> 위로 해드립니다']
['하루 가 또 가네요 <END>', '위로 해드립니다 <END>']


In [22]:
# 전체 문장을 하나의 문장으로 생성
all_sentences = questions + answer_in + answer_out

In [23]:
#토크나이저 와 수치화 및 패딩
import numpy as np
import warnings
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# WARNING 무시
warnings.filterwarnings('ignore')

# 토큰화
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(all_sentences)

# 텍스트를 시퀀스로 인코딩 (texts_to_sequences)
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

# 문장의 길이 맞추기 (pad_sequences)
MAX_LENGTH = 30
question_padded = pad_sequences(question_sequence, 
                                maxlen=MAX_LENGTH, 
                                truncating='post', 
                                padding='post')
answer_in_padded = pad_sequences(answer_in_sequence, 
                                 maxlen=MAX_LENGTH, 
                                 truncating='post', 
                                 padding='post')
answer_out_padded = pad_sequences(answer_out_sequence, 
                                  maxlen=MAX_LENGTH, 
                                  truncating='post', 
                                  padding='post')

In [24]:
# 단어 사전 확인
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t -> \t{idx}')
    if idx > 10:
        break

<OOV>	 -> 	1
<START>	 -> 	2
<END>	 -> 	3
이	 -> 	4
을	 -> 	5
가	 -> 	6
해보세요	 -> 	7
요	 -> 	8
보세요	 -> 	9
사람	 -> 	10
도	 -> 	11


In [25]:
# 토큰 개수 확인
len(tokenizer.word_index)

5360

In [26]:
#시퀀스 확인
question_padded.shape, answer_in_padded.shape, answer_out_padded.shape

((3001, 30), (3001, 30), (3001, 30))

In [27]:
#원핫 인코딩
VOCAB_SIZE = len(tokenizer.word_index)+1

#원핫 인코딩을 위한 함수
def convert_to_one_hot(padded):
    # 원핫인코딩 초기화
    one_hot_vector = np.zeros((len(answer_out_padded), 
                               MAX_LENGTH, 
                               VOCAB_SIZE))

    # 디코더 목표를 원핫 인코딩으로 변환
    # 학습시 입력은 인덱스이지만 출력은 원핫 인코딩 형식임
    for i, sequence in enumerate(answer_out_padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1

    return one_hot_vector

answer_in_one_hot = convert_to_one_hot(answer_in_padded)
answer_out_one_hot = convert_to_one_hot(answer_out_padded)
answer_in_one_hot[0].shape, answer_in_one_hot[0].shape

((30, 5361), (30, 5361))

In [28]:
# 변환된 index를 다시 단어로 변환
def convert_index_to_text(indexs, end_token): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == end_token:
            # 끝 단어이므로 예측 중비
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''
            
        # 빈칸 추가
        sentence += ' '
    return sentence

## 모델 생성

In [29]:
# 라이브러리 로드
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import plot_model

In [30]:
#입력을 위한 클래스 - Attention 사용
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps, 
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        # (attention) return_sequences=True 추가
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                         name='LSTM')
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # (attention) x return 추가
        return x, [hidden_state, cell_state]

In [31]:
# 출력을 위한 디코더
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps, 
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                         name='LSTM'
                        )
        self.attention = Attention(name='Attention')
        self.dense = Dense(vocab_size, 
                           activation='softmax', 
                           name='Dense')
    
    def call(self, inputs, initial_state):
        # (attention) encoder_inputs 추가
        encoder_inputs, decoder_inputs = inputs
        x = self.embedding(decoder_inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        
        # (attention) key_value, attention_matrix 추가
        # 이전 hidden_state의 값을 concat으로 만들어 vector를 생성합니다.        
        key_value = tf.concat([initial_state[0][:, tf.newaxis, :], 
                               x[:, :-1, :]], axis=1)        
        # 이전 hidden_state의 값을 concat으로 만든 vector와 encoder에서 나온 
        # 출력 값들로 attention을 구합니다.
        attention_matrix = self.attention([key_value, encoder_inputs])
        # 위에서 구한 attention_matrix와 decoder의 출력 값을 concat 합니다.
        x = tf.concat([x, attention_matrix], axis=-1)
        
        x = self.dense(x)
        return x, hidden_state, cell_state

In [32]:
class ChatModel(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(ChatModel, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps
        
        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)
        
        
    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(encoder_inputs)
            # (attention) decoder 입력 값 수정
            decoder_outputs, _, _ = self.decoder((encoder_outputs, decoder_inputs), 
                                                 initial_state=context_vector)
            return decoder_outputs
        else:
            x = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(x)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)
            
            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder((encoder_outputs, target_seq), 
                                                                            initial_state=context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)
                
                if decoder_output == self.end_token:
                    break
                    
                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]
                
            return tf.reshape(results.stack(), shape=(1, self.time_steps))

## 모델 생성

In [33]:
# 하이퍼 파라미터 설정
BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH
START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']

UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index)+1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3
NUM_EPOCHS = 20

In [34]:
# 체크 포인트 생성
checkpoint_path = 'model/chatmodel-attention-checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True,
                             save_best_only=True, 
                             monitor='loss', 
                             verbose=1
                            )

In [35]:
#모델 생성
chat = ChatModel(UNITS, 
                  VOCAB_SIZE, 
                  EMBEDDING_DIM, 
                  TIME_STEPS, 
                  START_TOKEN, 
                  END_TOKEN)

chat.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['acc'])

In [36]:
# 예측을 위한 함수
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

## 모델 학습

In [37]:
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    chat.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=10,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_padded[idx]
        # 문장 예측
        results = make_prediction(chat, np.expand_dims(question_inputs, 0))
        
        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)
        
        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
187/188 [============================>.] - ETA: 0s - loss: 1.8283 - acc: 0.8215
Epoch 1: loss improved from inf to 1.82682, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 13s 53ms/step - loss: 1.8268 - acc: 0.8215
Epoch 2/10
187/188 [============================>.] - ETA: 0s - loss: 1.0596 - acc: 0.8552
Epoch 2: loss improved from 1.82682 to 1.05972, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 11s 57ms/step - loss: 1.0597 - acc: 0.8552
Epoch 3/10
187/188 [============================>.] - ETA: 0s - loss: 0.9868 - acc: 0.8585
Epoch 3: loss improved from 1.05972 to 0.98675, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 11s 60ms/step - loss: 0.9867 - acc: 0.8585
Epoch 4/10
187/188 [============================>.] - ETA: 0s - loss: 0.9459 - acc: 0.8600
Epoch 4: loss improved from 0.98675 to 0.

188/188 [==============================] - ETA: 0s - loss: 0.3026 - acc: 0.9465
Epoch 9: loss improved from 0.31638 to 0.30261, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 13s 69ms/step - loss: 0.3026 - acc: 0.9465
Epoch 10/10
187/188 [============================>.] - ETA: 0s - loss: 0.2901 - acc: 0.9488
Epoch 10: loss improved from 0.30261 to 0.29002, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 13s 69ms/step - loss: 0.2900 - acc: 0.9488
Q: 병원 너무 가기 싫다
A: 맛있게 드세요 


Q: 나 는 왜 이렇게 태어났을까
A: 저 도 궁금하네요 


Q: 나 교직이수 할수있을까
A: 제 가 있잖아요 


processing epoch: 31...
Epoch 1/10
188/188 [==============================] - ETA: 0s - loss: 0.2795 - acc: 0.9504
Epoch 1: loss improved from 0.29002 to 0.27953, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 14s 72ms/step - loss: 0.2795 - acc: 0.9504
Epoch 2/10
188/188 [=====================

188/188 [==============================] - 23s 123ms/step - loss: 0.1557 - acc: 0.9653
Epoch 7/10
188/188 [==============================] - ETA: 0s - loss: 0.1536 - acc: 0.9657
Epoch 7: loss improved from 0.15572 to 0.15363, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 17s 90ms/step - loss: 0.1536 - acc: 0.9657
Epoch 8/10
187/188 [============================>.] - ETA: 0s - loss: 0.1502 - acc: 0.9655
Epoch 8: loss improved from 0.15363 to 0.15019, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 14s 76ms/step - loss: 0.1502 - acc: 0.9655
Epoch 9/10
188/188 [==============================] - ETA: 0s - loss: 0.1472 - acc: 0.9660
Epoch 9: loss improved from 0.15019 to 0.14725, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 14s 72ms/step - loss: 0.1472 - acc: 0.9660
Epoch 10/10
188/188 [==============================] - ETA: 0s -

188/188 [==============================] - 23s 121ms/step - loss: 0.0884 - acc: 0.9760
Epoch 5/10
188/188 [==============================] - ETA: 0s - loss: 0.0857 - acc: 0.9762
Epoch 5: loss improved from 0.08836 to 0.08566, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 18s 94ms/step - loss: 0.0857 - acc: 0.9762
Epoch 6/10
188/188 [==============================] - ETA: 0s - loss: 0.0841 - acc: 0.9768
Epoch 6: loss improved from 0.08566 to 0.08409, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 15s 82ms/step - loss: 0.0841 - acc: 0.9768
Epoch 7/10
188/188 [==============================] - ETA: 0s - loss: 0.0816 - acc: 0.9776
Epoch 7: loss improved from 0.08409 to 0.08160, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 16s 84ms/step - loss: 0.0816 - acc: 0.9776
Epoch 8/10
188/188 [==============================] - ETA: 0s - 

188/188 [==============================] - 17s 90ms/step - loss: 0.0408 - acc: 0.9890
Epoch 4/10
188/188 [==============================] - ETA: 0s - loss: 0.0393 - acc: 0.9893
Epoch 4: loss improved from 0.04085 to 0.03927, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 19s 101ms/step - loss: 0.0393 - acc: 0.9893
Epoch 5/10
188/188 [==============================] - ETA: 0s - loss: 0.0383 - acc: 0.9894
Epoch 5: loss improved from 0.03927 to 0.03830, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 23s 125ms/step - loss: 0.0383 - acc: 0.9894
Epoch 6/10
188/188 [==============================] - ETA: 0s - loss: 0.0371 - acc: 0.9897
Epoch 6: loss improved from 0.03830 to 0.03708, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 23s 122ms/step - loss: 0.0371 - acc: 0.9897
Epoch 7/10
188/188 [==============================] - ETA: 0s 

188/188 [==============================] - ETA: 0s - loss: 0.0182 - acc: 0.9953
Epoch 4: loss did not improve from 0.01746
188/188 [==============================] - 23s 122ms/step - loss: 0.0182 - acc: 0.9953
Epoch 5/10
188/188 [==============================] - ETA: 0s - loss: 0.0175 - acc: 0.9957
Epoch 5: loss did not improve from 0.01746
188/188 [==============================] - 20s 105ms/step - loss: 0.0175 - acc: 0.9957
Epoch 6/10
188/188 [==============================] - ETA: 0s - loss: 0.0157 - acc: 0.9958
Epoch 6: loss improved from 0.01746 to 0.01571, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 15s 81ms/step - loss: 0.0157 - acc: 0.9958
Epoch 7/10
188/188 [==============================] - ETA: 0s - loss: 0.0152 - acc: 0.9962
Epoch 7: loss improved from 0.01571 to 0.01521, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 15s 82ms/step - loss: 0.0152 - acc: 0.9962
Epo

188/188 [==============================] - 16s 84ms/step - loss: 0.0064 - acc: 0.9985
Epoch 6/10
188/188 [==============================] - ETA: 0s - loss: 0.0058 - acc: 0.9987
Epoch 6: loss improved from 0.00644 to 0.00581, saving model to model\chatmodel-attention-checkpoint.ckpt
188/188 [==============================] - 17s 90ms/step - loss: 0.0058 - acc: 0.9987
Epoch 7/10
188/188 [==============================] - ETA: 0s - loss: 0.0063 - acc: 0.9985
Epoch 7: loss did not improve from 0.00581
188/188 [==============================] - 19s 103ms/step - loss: 0.0063 - acc: 0.9985
Epoch 8/10
188/188 [==============================] - ETA: 0s - loss: 0.0069 - acc: 0.9983
Epoch 8: loss did not improve from 0.00581
188/188 [==============================] - 23s 125ms/step - loss: 0.0069 - acc: 0.9983
Epoch 9/10
188/188 [==============================] - ETA: 0s - loss: 0.0076 - acc: 0.9982
Epoch 9: loss did not improve from 0.00581
188/188 [==============================] - 22s 116ms/st

## 배포

In [38]:
# 자연어 (질문 입력) 대한 전처리 함수
def make_question(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

make_question('오늘 날씨 어때?')

array([[   1, 4364,  320,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [39]:
# 질문에 대한 답변을 리턴해주는 함수
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(chat, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

In [40]:
# 콘솔에서 테스트
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))

<< 말을 걸어 보세요!
배고프다
>> 챗봇 응답: 저 도밥 먹고싶어요 
<< 말을 걸어 보세요!
친구가 별로 없네~
>> 챗봇 응답: 제 가 따라가려면 멀었네요 
<< 말을 걸어 보세요!
집에 가고 싶다.
>> 챗봇 응답: 축하 드려요 


KeyboardInterrupt: Interrupted by user

In [ ]:
# 웹 서비스를 위한 배포
from flask import Flask, request
from flask import jsonify

app = Flask(__name__)

#시작 요청이 왔을 때 아래 코드를 수행
@app.route('/', methods=['POST', 'GET'])
def main():
    return 'Hello Chatbot'

@app.route('/chatbot', methods=['POST', 'GET'])
def chatbot():
    print(request.args["question"])
    answer = run_chatbot(request.args["question"])
    print('>> 챗봇 응답: {}'.format(answer))
    response = {'answer': answer}
    return jsonify(response)

#서버 구동
app.run('0.0.0.0', port=9000, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.20.16:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2022 16:58:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2022 16:58:19] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2022 16:58:52] "GET /chatbot?question=12 HTTP/1.1" 200 -


12
>> 챗봇 응답: 독서 와 음악 감상 이라고 하고싶지만 아무 것 도안 했어요 


127.0.0.1 - - [27/Apr/2022 17:58:43] "GET /chatbot?question=12 HTTP/1.1" 200 -


12
>> 챗봇 응답: 독서 와 음악 감상 이라고 하고싶지만 아무 것 도안 했어요 


...

알라딘이 뭐니?
>> 챗봇 응답: 얼른실내로 들어가세요 


127.0.0.1 - - [27/Apr/2022 17:59:22] "GET /chatbot?question=배고파요 HTTP/1.1" 200 -


배고파요
>> 챗봇 응답: 독서 와 음악 감상 이라고 하고싶지만 아무 것 도안 했어요 


127.0.0.1 - - [27/Apr/2022 17:59:31] "GET /chatbot?question=날씨가+좋아요~ HTTP/1.1" 200 -


날씨가 좋아요~
>> 챗봇 응답: 시도 해봐도 좋겠죠 
